In [40]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # -> 112x112
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)   # -> 56x56
        )
        self.gap = nn.AdaptiveAvgPool2d((1, 1))  # -> [B, 64, 1, 1]
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.gap(x)
        x = self.fc(x)
        return x


In [41]:
from torchvision.models import resnet18

model_tl = resnet18(weights="IMAGENET1K_V1")
for param in model_tl.parameters():
    param.requires_grad = False

# Replace classifier
model_tl.fc = nn.Linear(model_tl.fc.in_features, 10)


In [42]:
import time

def train_and_evaluate(model, name, trainloader, testloader, device, epochs=1):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(
        filter(lambda p: p.requires_grad, model.parameters()), 
        lr=0.01, momentum=0.9
    )

    print(f"\n🔹 Training {name}")
    start = time.time()
    model.train()
    for epoch in range(epochs):
        for x, y in trainloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            optimizer.step()
    end = time.time()
    print(f"⏱️ Training time: {end - start:.2f}s")

    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = correct / total * 100
    print(f"🎯 Accuracy: {acc:.2f}%")


In [43]:
# CIFAR-10
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Train both
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_and_evaluate(SimpleCNN(), "Simple CNN (scratch)", trainloader, testloader, device)
train_and_evaluate(model_tl, "Transfer Learning (ResNet18)", trainloader, testloader, device)


Files already downloaded and verified
Files already downloaded and verified

🔹 Training Simple CNN (scratch)


KeyboardInterrupt: 

Yes, exactly! You're using a ResNet18 pretrained on ImageNet as a feature extractor, and you're applying it to a downstream task: CIFAR-10 classification.

In [ ]:
for name, param in model_ft.named_parameters():
    param.requires_grad = ("layer4" in name) or ("fc" in name)


In [ ]:
train_and_evaluate(model_ft, "Fine-tuned ResNet18", trainloader, testloader, device, epochs=5)